# Coronavirus Data Analysis

In [2]:
import pandas as pd
import numpy as np

In [3]:
spark.sparkContext.applicationId

'application_1636170550178_0190'

In [4]:
pd.options.display.max_columns = 35

In [5]:
spark.stop()

spark = SparkSession.builder \
                    .config('spark.dynamicAllocation.maxExecutors', 50) \
                    .config('spark.executor.cores', '3') \
                    .config('spark.executor.memory', '15g') \
                    .config('spark.driver.memory', '40g') \
                    .config('spark.python.worker.memory', '2g') \
                    .config('spark.sql.shuffle.partitions', '2000') \
                    .config('spark.network.timeout', '2000') \
                    .config('spark.sql.broadcastTimeout', '2000') \
                    .config('spark.driver.maxResultSize', '6g') \
                    .config('mapreduce.fileoutputcommitter.algorithm.version', '2') \
                    .getOrCreate()
            
spark.sparkContext.applicationId

'application_1636170550178_0192'

In [6]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831', '6E70BFD2F17848BBB19A31ECDFB49EE7'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831
6E70BFD2F17848BBB19A31ECDFB49EE7


In [7]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


In [8]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


### Create Variables for Queries

In [10]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [9]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [10]:
spark.sql(f"""select * FROM {userName}.{algorithmName}_Hospital h  """).printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- tenant_bed_size: string (nullable = true)
 |-- tenant_speciality: string (nullable = true)
 |-- tenant_segment: string (nullable = true)
 |-- tenant_zip_code: string (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- dischargeDate: string (nullable = true)
 |-- length_of_stay: integer (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- deceased: integer (nullable = true)
 |-- discharged_to_skilled_nursing_facility: integer (nullable = true)
 |-- discharged_to_hospice: integer (nullable = true)
 |-- died_in_the_hospital: integer (nullable = true)
 |-- discharged_to_rehab_facility: integer (nullable = true)
 |-- discharged_to_long_term_care_facility: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- gender: string (nullable = true)



In [11]:
spark.sql("""select * from lab """).printSchema()

root
 |-- labid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- labcode: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- loincclass: string (nullable = true)
 |-- type: string (nullable = true)
 |-- servicedate: string (nullable = true)
 |-- serviceperiod: struct (nullable = true)
 |    |-- startDate: string (nullable = true)
 |    |-- endDate: string (nullable = true)
 |-- typedvalue: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- textValue: 

In [12]:
spark.sql(f"""


    SELECT distinct 
        h.tenant
        ,count(distinct h.personId)
        
            FROM {userName}.{algorithmName}_Hospital h  
            
  group by 1          

  order by 1
    


""").toPandas()

,tenant,count(DISTINCT personId)
0,1,1862
1,2,30
2,3,1340
3,4,18
4,5,130
...,...,...
91,98,10
92,99,255
93,100,205
94,101,471


In [13]:
spark.sql(f"""
select distinct l.type
        
     FROM {db}.lab l
         WHERE NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                
""").toPandas()

,type
0,TEXT
1,DATE
2,NUMERIC
3,CODIFIED


In [17]:
commonlabsDf = spark.sql(f"""
   select l.labcode.standard.id as labcode_id
         ,l.labcode.standard.primaryDisplay as labcode_display
         ,l.labcode.standard.codingSystemId as labcode_system_id
         ,count(distinct h.personId) as count
       


  
        FROM {userName}.{algorithmName}_Hospital h  
        

       

        JOIN {db}.lab l
            ON  h.encounterId = l.encounterId
                and h.personId = l.personID
                and h.tenant = l.tenant
 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                          
          
     AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, 
                    array('COVID_19_EVALUATION_NOTE',
'COVID_19_INITIAL_EVALUATION_FORM',
'EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM',
'INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
'INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY',
'INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY',
'MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA',
'POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS',
'RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE',
'SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION',
'SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION',
'SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA',
'SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY',
'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY',
'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT',
'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE',
'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING',
'SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT',
'SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2',
'SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING',
'SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE',
'SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD',
'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
'SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'

                      ), 
                      '6E70BFD2F17848BBB19A31ECDFB49EE7')  
                              
                
        
                               
         group by 1,2,3
         
         having count >= 100000
          
          
         order by 4 desc 
         
      
         
   
  
         
""")

In [18]:
#commonlabsDf.toPandas()

In [32]:
##### Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_LabSummary""").show() 

++
||
++
++



In [33]:
commonlabsDf.write.saveAsTable(f"""{userName}.{algorithmName}_LabSummary """)

In [19]:
spark.sql(f"""select * from {userName}.{algorithmName}_LabSummary """).toPandas().to_excel("test.xlsx")

In [18]:
labResultsDf = spark.sql(f"""
   select h.tenant
         ,h.personId
         ,h.encounterId
         ,l.labid
         ,l.serviceDate
         ,l.labcode.standard.id as labcode_id
         ,l.labcode.standard.primaryDisplay as labcode_display
         ,l.labcode.standard.codingSystemId as labcode_system_id
         ,l.type
         ,case  
           when l.type = 'NUMERIC' then cast(typedvalue.numericValue.value as string)
           when l.type = 'TEXT' then cast(typedvalue.textvalue.value as string)
           when l.type = 'DATE' then cast(typedvalue.datevalue.date as string)
           else cast(typedvalue.codifiedValues.values.value.standard.primaryDisplay as string)  
          end labcode_value   
         ,l.unitofmeasure.standard.id as labcode_unitofmeasure_id
         ,l.unitofmeasure.standard.primaryDisplay as labcode_unitofmeasure_display
         ,l.unitofmeasure.standard.codingSystemId as labcode_unitofmeasure_system_id 
         ,l.interpretation.standard.id as labcode_interpretation_id
         ,l.interpretation.standard.primaryDisplay as labcode_interpretation_display
         ,l.interpretation.standard.codingSystemId as labcode_interpretation_system_id 
   

  
        FROM {userName}.{algorithmName}_Hospital h  
        

       

        JOIN {db}.lab l
            ON  h.encounterId = l.encounterId
                and h.personId = l.personID
                and h.tenant = l.tenant
 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                          
          
  
       AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, 
                    array('COVID_19_EVALUATION_NOTE',
'COVID_19_INITIAL_EVALUATION_FORM',
'EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM',
'INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
'INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY',
'INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY',
'MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA',
'POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS',
'RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE',
'SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION',
'SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION',
'SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA',
'SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY',
'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY',
'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT',
'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE',
'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING',
'SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT',
'SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2',
'SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING',
'SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE',
'SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD',
'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
'SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'

                      ), 
                      '6E70BFD2F17848BBB19A31ECDFB49EE7')  
                
      
                               
      --   group by 1,2,3
      --   order by 4 desc 

         
""")

In [19]:
# labResultsDf.toPandas()

In [20]:
# Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_DetailedPatientsLabs""").show() 

++
||
++
++



In [21]:
labResultsDf.write.saveAsTable(f"""{userName}.{algorithmName}_DetailedPatientsLabs """)

In [47]:
labResultsTransposeDf = spark.sql(f"""
 select distinct l.tenant 
                ,l.personId
                ,l.encounterId
                ,l.serviceDate
     ,MAX(IF(l.labcode_id in ('48642-3', '48643-1'), l.labcode_id, null)) as taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_id
     ,MAX(IF(l.labcode_id='1920-8', l.labcode_id, null)) as taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='1742-6', l.labcode_id, null)) as taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='6768-6', l.labcode_id, null)) as taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='32623-1', l.labcode_id, null)) as taken_PMV_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='4544-3', l.labcode_id, null)) as taken_HCT_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='788-0', l.labcode_id, null)) as taken_RDW_RATIO_AUTO_lab_id
     ,MAX(IF(l.labcode_id='2028-9', l.labcode_id, null)) as taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='713-8', l.labcode_id, null)) as taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='736-9', l.labcode_id, null)) as taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='5905-5', l.labcode_id, null)) as taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='706-2', l.labcode_id, null)) as taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='1988-5', l.labcode_id, null)) as taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='789-8', l.labcode_id, null)) as taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='6690-2', l.labcode_id, null)) as taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='731-0', l.labcode_id, null)) as taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='751-8', l.labcode_id, null)) as taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='711-2', l.labcode_id, null)) as taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='1975-2', l.labcode_id, null)) as taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='777-3', l.labcode_id, null)) as taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='704-7', l.labcode_id, null)) as taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='742-7', l.labcode_id, null)) as taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_id
     ,MAX(IF(l.labcode_id='3094-0', l.labcode_id, null)) as taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='2160-0', l.labcode_id, null)) as taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='2823-3', l.labcode_id, null)) as taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='19123-9', l.labcode_id, null)) as taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='2075-0', l.labcode_id, null)) as taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='17861-6', l.labcode_id, null)) as taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='2345-7', l.labcode_id, null)) as taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='2951-2', l.labcode_id, null)) as taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='2885-2', l.labcode_id, null)) as taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='1751-7', l.labcode_id, null)) as taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_id
     ,MAX(IF(l.labcode_id='787-2', l.labcode_id, null)) as taken_MCV_AUTO_lab_id
     ,MAX(IF(l.labcode_id='786-4', l.labcode_id, null)) as taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_id
     ,MAX(IF(l.labcode_id='785-6', l.labcode_id, null)) as taken_MCH_AUTO_lab_id
     ,MAX(IF(l.labcode_id='718-7', l.labcode_id, null)) as taken_HGB_MASSVOLUME_BLOOD_lab_id
     ,MAX(IF(l.labcode_id='5902-2', l.labcode_id, null)) as taken_PROTHROMBIN_TIME_lab_id
     ,MAX(IF(l.labcode_id='5778-6', l.labcode_id, null)) as taken_COLOR_URINE_lab_id


     ,MAX(IF(l.labcode_id in ('48642-3', '48643-1'), 1,0)) as taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_test
     ,MAX(IF(l.labcode_id='1920-8', 1,0)) as taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='1742-6', 1,0)) as taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='6768-6', 1,0)) as taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='32623-1', 1,0)) as taken_PMV_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='4544-3', 1,0)) as taken_HCT_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='788-0', 1,0)) as taken_RDW_RATIO_AUTO_lab_test
     ,MAX(IF(l.labcode_id='2028-9', 1,0)) as taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='713-8', 1,0)) as taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='736-9', 1,0)) as taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='5905-5', 1,0)) as taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_test
       ,MAX(IF(l.labcode_id='706-2', 1,0)) as taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='1988-5', 1,0)) as taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='789-8', 1,0)) as taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='6690-2', 1,0)) as taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='731-0', 1,0)) as taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='751-8', 1,0)) as taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='711-2', 1,0)) as taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='1975-2', 1,0)) as taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='777-3', 1,0)) as taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='704-7', 1,0)) as taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='742-7', 1,0)) as taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_test
     ,MAX(IF(l.labcode_id='3094-0', 1,0)) as taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='2160-0', 1,0)) as taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='2823-3', 1,0)) as taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='19123-9', 1,0)) as taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='2075-0', 1,0)) as taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='17861-6', 1,0)) as taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='2345-7', 1,0)) as taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='2951-2', 1,0)) as taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='2885-2', 1,0)) as taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='1751-7', 1,0)) as taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_test
     ,MAX(IF(l.labcode_id='787-2', 1,0)) as taken_MCV_AUTO_lab_test
     ,MAX(IF(l.labcode_id='786-4', 1,0)) as taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_test
     ,MAX(IF(l.labcode_id='785-6', 1,0)) as taken_MCH_AUTO_lab_test
     ,MAX(IF(l.labcode_id='718-7', 1,0)) as taken_HGB_MASSVOLUME_BLOOD_lab_test
     ,MAX(IF(l.labcode_id='5902-2', 1,0)) as taken_PROTHROMBIN_TIME_lab_test
     ,MAX(IF(l.labcode_id='5778-6', 1,0)) as taken_COLOR_URINE_lab_test


     ,MAX(IF(l.labcode_id in ('48642-3', '48643-1'), labcode_value,null)) as taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_value
     ,MAX(IF(l.labcode_id='1920-8', labcode_value,null)) as taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='1742-6', labcode_value,null)) as taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='6768-6', labcode_value,null)) as taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='32623-1', labcode_value,null)) as taken_PMV_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='4544-3', labcode_value,null)) as taken_HCT_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='788-0', labcode_value,null)) as taken_RDW_RATIO_AUTO_lab_value
     ,MAX(IF(l.labcode_id='2028-9', labcode_value,null)) as taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='713-8', labcode_value,null)) as taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='736-9', labcode_value,null)) as taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='5905-5', labcode_value,null)) as taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='706-2', labcode_value,null)) as taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='1988-5', labcode_value,null)) as taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='789-8', labcode_value,null)) as taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='6690-2', labcode_value,null)) as taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='731-0', labcode_value,null)) as taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='751-8', labcode_value,null)) as taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='711-2', labcode_value,null)) as taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='1975-2', labcode_value,null)) as taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='777-3', labcode_value,null)) as taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='704-7', labcode_value,null)) as taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='742-7', labcode_value,null)) as taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_value
     ,MAX(IF(l.labcode_id='3094-0', labcode_value,null)) as taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='2160-0', labcode_value,null)) as taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='2823-3', labcode_value,null)) as taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='19123-9', labcode_value,null)) as taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='2075-0', labcode_value,null)) as taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='17861-6', labcode_value,null)) as taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='2345-7', labcode_value,null)) as taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='2951-2', labcode_value,null)) as taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='2885-2', labcode_value,null)) as taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='1751-7', labcode_value,null)) as taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_value
     ,MAX(IF(l.labcode_id='787-2', labcode_value,null)) as taken_MCV_AUTO_lab_value
     ,MAX(IF(l.labcode_id='786-4', labcode_value,null)) as taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_value
     ,MAX(IF(l.labcode_id='785-6', labcode_value,null)) as taken_MCH_AUTO_lab_value
     ,MAX(IF(l.labcode_id='718-7', labcode_value,null)) as taken_HGB_MASSVOLUME_BLOOD_lab_value
     ,MAX(IF(l.labcode_id='5902-2', labcode_value,null)) as taken_PROTHROMBIN_TIME_lab_value
     ,MAX(IF(l.labcode_id='5778-6', labcode_value,null)) as taken_COLOR_URINE_lab_value


         ,MAX(IF(l.labcode_id in ('48642-3', '48643-1'),labcode_interpretation_display,null)) as taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_interpretation
     ,MAX(IF(l.labcode_id='1920-8',labcode_interpretation_display,null)) as taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='1742-6',labcode_interpretation_display,null)) as taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='6768-6',labcode_interpretation_display,null)) as taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='32623-1',labcode_interpretation_display,null)) as taken_PMV_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='4544-3',labcode_interpretation_display,null)) as taken_HCT_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='788-0',labcode_interpretation_display,null)) as taken_RDW_RATIO_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='2028-9',labcode_interpretation_display,null)) as taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='713-8',labcode_interpretation_display,null)) as taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='736-9',labcode_interpretation_display,null)) as taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='5905-5',labcode_interpretation_display,null)) as taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='706-2',labcode_interpretation_display,null)) as taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='1988-5',labcode_interpretation_display,null)) as taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='789-8',labcode_interpretation_display,null)) as taken_ERYTHROCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='6690-2',labcode_interpretation_display,null)) as taken_LEUKOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='731-0',labcode_interpretation_display,null)) as taken_LYMPHOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='751-8',labcode_interpretation_display,null)) as taken_NEUTROPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='711-2',labcode_interpretation_display,null)) as taken_EOSINOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='1975-2',labcode_interpretation_display,null)) as taken_BILIRUBIN_TOTAL_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='777-3',labcode_interpretation_display,null)) as taken_PLATELETS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='704-7',labcode_interpretation_display,null)) as taken_BASOPHILS_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='742-7',labcode_interpretation_display,null)) as taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='3094-0',labcode_interpretation_display,null)) as taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='2160-0',labcode_interpretation_display,null)) as taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='2823-3',labcode_interpretation_display,null)) as taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='19123-9',labcode_interpretation_display,null)) as taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='2075-0',labcode_interpretation_display,null)) as taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='17861-6',labcode_interpretation_display,null)) as taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='2345-7',labcode_interpretation_display,null)) as taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='2951-2',labcode_interpretation_display,null)) as taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='2885-2',labcode_interpretation_display,null)) as taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='1751-7',labcode_interpretation_display,null)) as taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation
     ,MAX(IF(l.labcode_id='787-2',labcode_interpretation_display,null)) as taken_MCV_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='786-4',labcode_interpretation_display,null)) as taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='785-6',labcode_interpretation_display,null)) as taken_MCH_AUTO_lab_interpretation
     ,MAX(IF(l.labcode_id='718-7',labcode_interpretation_display,null)) as taken_HGB_MASSVOLUME_BLOOD_lab_interpretation
     ,MAX(IF(l.labcode_id='5902-2',labcode_interpretation_display,null)) as taken_PROTHROMBIN_TIME_lab_interpretation
     ,MAX(IF(l.labcode_id='5778-6',labcode_interpretation_display,null)) as taken_COLOR_URINE_lab_interpretation


 from {userName}.{algorithmName}_DetailedPatientsLabs l
   
   where l.labcode_id in ('48642-3',
                    '48643-1',
                    '1920-8',
                    '1742-6',
                    '6768-6',
                    '32623-1',
                    '4544-3',
                    '788-0',
                    '2028-9',
                    '713-8',
                    '736-9',
                    '5905-5',
                    '706-2',
                    '1988-5',
                    '789-8',
                    '6690-2',
                    '731-0',
                    '751-8',
                    '711-2',
                    '1975-2',
                    '777-3',
                    '704-7',
                    '742-7',
                    '3094-0',
                    '2160-0',
                    '2823-3',
                    '19123-9',
                    '2075-0',
                    '17861-6',
                    '2345-7',
                    '2951-2',
                    '2885-2',
                    '1751-7',
                    '787-2',
                    '786-4',
                    '785-6',
                    '718-7',
                    '5902-2',
                    '5778-6'
                    )
   
   group by 1,2,3,4
   order by 1,2,3,4
   
""")



In [48]:
labResultsTransposeDf.limit(5).toPandas()

,tenant,personId,encounterId,serviceDate,taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_id,taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id,taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id,taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id,taken_PMV_BLOOD_AUTO_lab_id,taken_HCT_BLOOD_AUTO_lab_id,taken_RDW_RATIO_AUTO_lab_id,taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_id,taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_id,taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_id,taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_id,taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_id,taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_id,...,taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation,taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_MCV_AUTO_lab_interpretation,taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation,taken_MCH_AUTO_lab_interpretation,taken_HGB_MASSVOLUME_BLOOD_lab_interpretation,taken_PROTHROMBIN_TIME_lab_interpretation,taken_COLOR_URINE_lab_interpretation
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T00:00:00+00:00,48642-3,1920-8,None,6768-6,32623-1,4544-3,788-0,2028-9,None,None,None,None,None,...,None,Normal,Normal,Normal,None,Normal,Normal,Normal,Below low normal,Normal,None,Normal,Normal,Normal,Below low normal,Normal,None
1,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T00:08:00+00:00,48642-3,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T11:53:00+00:00,48642-3,None,None,None,None,None,None,2028-9,None,None,None,None,None,...,None,Normal,Below low normal,Normal,Normal,Normal,Normal,Normal,Normal,None,None,None,None,None,None,None,None
3,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T12:12:00+00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-12T10:00:00+00:00,None,None,None,None,32623-1,4544-3,788-0,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,Normal,Normal,Normal,Below low normal,None,None


In [49]:
# Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_SummaryPatientsLabs""").show() 

++
||
++
++



In [50]:
labResultsTransposeDf.write.saveAsTable(f"""{userName}.{algorithmName}_SummarydPatientsLabs """)

In [11]:
spark.sql(f"""select count(*)
from {userName}.{algorithmName}_SummarydPatientsLabs
""").show()

+--------+
|count(1)|
+--------+
| 3869806|
+--------+



In [12]:
spark.sql(f"""select * 
from {userName}.{algorithmName}_SummarydPatientsLabs
order by 1, 2, 3
limit(25)
""").toPandas()

,tenant,personId,encounterId,serviceDate,taken_ESTIMATED_GLOMERULAR_FILTRATION_RATE_OBSTYPE_lab_id,taken_ASPARTATE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id,taken_ALANINE_AMINOTRANSFERASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id,taken_ALKALINE_PHOSPHATASE_ENZYMATIC_ACTIVITY_VOLUME_SERUM_PLASMA_lab_id,taken_PMV_BLOOD_AUTO_lab_id,taken_HCT_BLOOD_AUTO_lab_id,taken_RDW_RATIO_AUTO_lab_id,taken_CO2_TOTAL_MOLESVOLUME_SERUM_PLASMA_lab_id,taken_EOSINOPHILS_PERCENT_BLOOD_AUTO_lab_id,taken_LYMPHOCYTES_PERCENT_BLOOD_AUTO_lab_id,taken_MONOCYTES_PERCENT_BLOOD_AUTO_lab_id,taken_BASOPHILS_PERCENT_BLOOD_AUTO_lab_id,taken_C_REACTIVE_PROTIEN_MASSVOLUMNE_BLOOD_AUTO_lab_id,...,taken_MONOCYTES_NUMBERVOLUME_BLOOD_AUTO_lab_interpretation,taken_BUN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_CREATININE_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_POTASSIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_MAGNESIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_CHLORIDE_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_CALCIUM_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_GLUCOSE_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_SODIUM_MOLESVOLUME_SERUM_PLASMA_lab_interpretation,taken_PROTEIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_ALBUMIN_MASSVOLUME_SERUM_PLASMA_lab_interpretation,taken_MCV_AUTO_lab_interpretation,taken_MCHC_MASSVOLUME_VOLUME_AUTO_lab_interpretation,taken_MCH_AUTO_lab_interpretation,taken_HGB_MASSVOLUME_BLOOD_lab_interpretation,taken_PROTHROMBIN_TIME_lab_interpretation,taken_COLOR_URINE_lab_interpretation
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-12T10:00:00+00:00,None,None,None,None,32623-1,4544-3,788-0,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,Normal,Normal,Normal,Below low normal,None,None
1,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-12T15:06:00+00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-14T13:53:00+00:00,48642-3,1920-8,None,6768-6,32623-1,4544-3,788-0,2028-9,None,None,None,None,None,...,None,Normal,Below low normal,Normal,None,Normal,Normal,Normal,Normal,Below low normal,None,Normal,Normal,Normal,Below low normal,None,None
3,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T12:12:00+00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T11:53:00+00:00,48642-3,None,None,None,None,None,None,2028-9,None,None,None,None,None,...,None,Normal,Below low normal,Normal,Normal,Normal,Normal,Normal,Normal,None,None,None,None,None,None,None,None
5,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-13T23:45:10+00:00,48642-3,1920-8,None,6768-6,None,None,None,2028-9,None,None,None,None,None,...,None,Below low normal,Below low normal,Normal,None,Normal,Normal,Above high normal,Normal,Normal,None,None,None,None,None,None,None
6,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-15T13:29:26+00:00,48642-3,1920-8,None,6768-6,32623-1,4544-3,788-0,2028-9,None,None,None,None,None,...,None,Normal,Normal,Normal,None,Normal,Normal,Normal,Normal,Below low normal,None,Normal,Normal,Normal,Below low normal,None,None
7,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T00:00:00+00:00,48642-3,1920-8,None,6768-6,32623-1,4544-3,788-0,2028-9,None,None,None,None,None,...,None,Normal,Normal,Normal,None,Normal,Normal,Normal,Below low normal,Normal,None,Normal,Normal,Normal,Below low normal,Normal,None
8,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e